In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                                 options = [
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                                ],
                                                 value = "ALL",
                                                 placeholder = "Select a Launch Site here",
                                                 searchable = True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),


                                ###html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)

                                html.P("Payload range (Kg):",style={'textAlign': 'center', 'color': '#503D36','font-size': 24}),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={ 0: '0 Kg',
                                            1000: '1000 Kg',
                                            2000: '2000 Kg',
                                            3000: '3000 Kg',
                                            4000: '4000 Kg',
                                            5000: '5000 Kg',
                                            6000: '6000 Kg',
                                            7000: '7000 Kg',
                                            8000: '8000 Kg',
                                            9000: '9000 Kg',
                                            10000: '10000 Kg'},
                                    value=[min_payload, max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ###                              
                               ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    launch_sites = spacex_df["Launch Site"].unique()
    success_failure_dict = {}
    for i in range(len(launch_sites)):
        success_failure = spacex_df.loc[spacex_df["Launch Site"]==launch_sites[i],"class"].value_counts()
        success_failure_dict[launch_sites[i]] = success_failure
    launch_sites_success_failure_df = pd.DataFrame(success_failure_dict)
    launch_sites_success_failure_df["Outcome"] = ["Failure","Success"]
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Successful launches for all launch sites')
        return fig
    else:
        fig = px.pie(launch_sites_success_failure_df,
                     values=entered_site,
                     names="Outcome",
                     title='Successful launches from ' + entered_site
                    )
        fig.update_traces(sort=False)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
               [Input(component_id='site-dropdown', component_property='value'), 
                Input(component_id="payload-slider", component_property="value")])
def get_scatter(selected_launch_site,payload):
    #filtered_df = spacex_df.loc[(spacex_df["Payload Mass (kg)"]>=payload[0]) and (spacex_df["Payload Mass (kg)"]<=payload[1])]
    filtered_df = spacex_df.loc[spacex_df["Payload Mass (kg)"].between(payload[0],payload[1])]
    if (selected_launch_site == "ALL"):
        fig = px.scatter(x=filtered_df["Payload Mass (kg)"], 
                         y=filtered_df["class"],
                         color=filtered_df["Booster Version Category"],
                         labels={
                            "x": "Payload Mass (kg)",
                            "y": "Launch Success",
                            "color": "Booster Version"
                         },
                         title="Payload Mass vs Launch Success for all launch sites")
    else:
        filtered_df = filtered_df.loc[filtered_df["Launch Site"] == selected_launch_site]
        fig = px.scatter(x=filtered_df["Payload Mass (kg)"], 
                         y=filtered_df["class"],
                         color=filtered_df["Booster Version Category"],
                         labels={
                            "x": "Payload Mass (kg)",
                            "y": "Launch Success",
                            "color": "Booster Version"
                        },
                        title="Payload Mass vs Launch Success for "+selected_launch_site)
    return fig
# Run the app
if __name__ == '__main__':
    app.run_server(port=8051)


C:\Users\moawi\AppData\Local\Temp\ipykernel_23504\2728804763.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\moawi\AppData\Local\Temp\ipykernel_23504\2728804763.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
